# LangChain全面剖析之Chains

## 1. Chains的设计理念

### 1.1 Chains的设计理念

&emsp;&emsp;举个例子，网站登录模块的设计，这是一个常见的应用开发任务。在最简单的形式中，登录流程可能仅涉及用户输入手机号码并接收验证码以登录。这个过程看似直接，实际上却需要后台执行一系列操作，如验证手机号码的有效性、生成并发送验证码、以及最终验证用户输入的验证码是否正确。而更复杂的设计，登录模块就还需要进行额外的验证步骤，比如确认手机号码是否已经注册、是否与用户的账户信息匹配、甚至检查账户是否存在安全问题。在这种情况下，开发者可能需要设计更复杂的输入模板和逻辑流程。

&emsp;&emsp;所以能够预见到，即使是看似简单的登录模块，也包含了多个子逻辑的串联，每个步骤都需要经过精心设计的输入模板和输出选择，可以理解为每个子逻辑都会包含一个或多个Model I/O的设计。那么多个子逻辑如何去链接成一整个有效的通路，这就是LangChain中Chains抽象模块要解决的核心问题。

### 1.2 最简单的Chain

In [12]:
import openai
import os
from langchain_openai import ChatOpenAI
from openai import OpenAI
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.api_base="https://newone.nxykj.tech/v1"
openai.api_key = os.getenv("DEEPSEEK_API_KEY")
openai.api_base = "https://api.deepseek.com"
llm = ChatOpenAI(model_name="deepseek-chat",api_key=openai.api_key ,base_url=openai.api_base)

In [13]:
from langchain_core.prompts import ChatPromptTemplate

func_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个{language}代码编辑器，你的功能是根据自然语言的输入描述，输出对应的函数代码，无需输出任何解释，且务必不要输出任何与代码无关的内容"),
        ("human", "当前的题目要求是：{user_input}"),
    ]
)

In [14]:
func_chains = func_template | llm

In [15]:
func_chains_reponse = func_chains.invoke({"language":"java",
                                          "user_input":"给定一个整数数组 nums 和一个整数目标值 target，请你在该数组中找出 和为目标值 target  的那 两个 整数，并返回它们的数组下标。\
                                          你可以假设每种输入只会对应一个答案。但是，数组中同一个元素在答案里不能重复出现。你可以按任意顺序返回答案。"
                                            }
                                               ) 

In [16]:
print(func_chains_reponse.content)

```java
public int[] twoSum(int[] nums, int target) {
    Map<Integer, Integer> map = new HashMap<>();
    for (int i = 0; i < nums.length; i++) {
        int complement = target - nums[i];
        if (map.containsKey(complement)) {
            return new int[] { map.get(complement), i };
        }
        map.put(nums[i], i);
    }
    throw new IllegalArgumentException("No two sum solution");
}
```


## 2 LangChain提供的Chains的类型

### 2.1 基于LCEC语言构建的Chains的类型

```python
create_stuff_documents_chain
create_openai_fn_runnable
create_structured_output_runnable
load_query_constructor_runnable
create_sql_query_chain
create_history_aware_retriever
create_retrieval_chain
```

#### create_sql_query_chain

```python
# pip install -U langchain langchain-community langchain-openai
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
```

#### create_openai_fn_runnable

In [4]:
from typing import Optional

from langchain.chains.structured_output import create_openai_fn_runnable
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field

In [5]:
from typing import Optional
from pydantic import BaseModel, Field

class RecordPerson(BaseModel):
    '''记录关于一个人的一些识别信息。'''

    name: str = Field(..., description="这个人的名字")
    age: int = Field(..., description="这个人的年龄")
    fav_food: Optional[str] = Field(None, description="这个人最喜欢的食物")


class RecordDog(BaseModel):
    '''记录关于一只狗的一些识别信息。'''

    name: str = Field(..., description="这只狗的名字")
    color: str = Field(..., description="这只狗的颜色")
    fav_food: Optional[str] = Field(None, description="这只狗最喜欢的食物")

In [11]:
import openai
from langchain_openai import ChatOpenAI
from openai import OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base="https://newone.nxykj.tech/v1"

llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)

In [13]:
chain = create_openai_fn_runnable([RecordPerson, RecordDog], llm)
chain.invoke("哈里是一只胖乎乎的棕色比格犬，非常喜欢吃鸡肉。")

{'arguments': {'name': '哈里', 'color': '棕色'}, 'name': 'RecordDog'}

In [15]:
chain = create_openai_fn_runnable([RecordPerson, RecordDog], llm)
chain.invoke("我有一个朋友，今年19岁，名字叫王大美")

{'arguments': {'name': '王大美', 'age': 19}, 'name': 'RecordPerson'}

&emsp;&emsp;通过上面两个例子，我们可以看到，当输入与狗相关的Prompt时（哈里是一只胖乎乎的棕色比格犬，非常喜欢吃鸡肉。），该流程会执行`RecordDog`类的功能逻辑，自动记录狗的名字和颜色。相反，若输入涉及人的信息（我有一个朋友，今年19岁，名字叫王大美），则`RecordPerson`类的逻辑将被执行，实现对人的信息的自动记录。实现对不同输入的自动识别和相应外部函数的匹配逻辑。

In [16]:
from langchain_core.tools import tool

import requests
import json

In [17]:
open_weather_key = "5c939a7cc59eb8696f4cd77bf75c5a9a"

In [18]:
@tool
def get_weather(loc:str):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [19]:
functions = [get_weather]

In [20]:
from langchain_core.utils.function_calling import convert_to_openai_function

openai_functions = [convert_to_openai_function(f) for f in functions]  # 这行代码来源于create_openai_fn_runnable源码流程
openai_functions

[{'name': 'get_weather',
  'description': "get_weather(loc: str) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息",
  'parameters': {'type': 'object',
   'properties': {'loc': {'type': 'string'}},
   'required': ['loc']}}]

In [21]:
chain = create_openai_fn_runnable([get_weather], llm)
chain.invoke("北京今天的天气怎么样？")

{'loc': 'Beijing'}

In [23]:
chain = create_openai_fn_runnable([get_weather], llm)
chain.invoke("上海今天天气怎么样？")

{'loc': 'Shanghai'}

In [50]:
@tool
def getStockInfor(companyName: str) -> str:
    """
    获取股票信息
    :param companyName: 必要参数，字符串类型，用于表示被查询股票的公司名称。
    :return：股票查询的结果结果，
    """
    return companyName+"的股票今天走势非常好！"

In [51]:
functions = [get_weather, getStockInfor]

In [52]:
from langchain_core.utils.function_calling import convert_to_openai_function

openai_functions = [convert_to_openai_function(f) for f in functions]  # 这行代码来源于create_openai_fn_runnable源码流程
openai_functions

[{'name': 'get_weather',
  'description': "get_weather(loc: str) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息",
  'parameters': {'type': 'object',
   'properties': {'loc': {'type': 'string'}},
   'required': ['loc']}},
 {'name': 'getStockInfor',
  'description': 'getStockInfor(companyName: str) -> str - 获取股票信息\n    :param companyName: 必要参数，字符串类型，用于表示被查询股票的公司名称。\n    :return：股票查询的结果结果，',
  'parameters': {'type': 'object',
   'properties': {'companyName': {'type': 'string'}},
   'required': ['companyName']}}]

In [29]:
chain = create_openai_fn_runnable([get_weather, getStockInfor], llm)
chain.invoke("北京今天的天气怎么样？")

{'arguments': {'loc': 'Beijing'}, 'name': 'get_weather'}

In [53]:
chain = create_openai_fn_runnable([get_weather, getStockInfor], llm)
chain.invoke("亚马逊公司的今天股票如何？")

{'arguments': {'companyName': 'Amazon'}, 'name': 'getStockInfor'}

&emsp;&emsp;`itemgetter("arguments")`会解析经过`create_openai_fn_runnable([get_weather, multiply], llm)`输出的结果，识别到StructuredTool列表，而后如果再通过 | + 具体函数名称的方式，即可实现执行具体函数内部逻辑的功能。比如我们传入 `get_weather`，就是执行 `get_weather`函数去查询实时的天气。

In [33]:
from operator import itemgetter

chain = create_openai_fn_runnable([get_weather, getStockInfor], llm) | itemgetter("arguments") | get_weather
chain.invoke("北京今天的天气怎么样？")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 800, "main": "Clear", "description": "\\u6674", "icon": "01n"}], "base": "stations", "main": {"temp": 26.94, "feels_like": 25.9, "temp_min": 26.94, "temp_max": 26.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 999}, "visibility": 10000, "wind": {"speed": 5.54, "deg": 201, "gust": 12.43}, "clouds": {"all": 3}, "dt": 1715602742, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [56]:
chain = create_openai_fn_runnable([get_weather, getStockInfor], llm) | itemgetter("arguments") | getStockInfor
chain.invoke("亚马逊公司的今天股票如何？")

'Amazon的股票今天走势非常好！'

&emsp;&emsp;从结果上看，确实是能够得到函数执行的最终数据，那么这个过程，我们就可以封装成输出解析器（OutPut Parser），结合`create_openai_fn_runnable`返回最终函数执行的结果。输出解析器执行逻辑如下：

In [57]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import json

def final_resonse(ai_message: AIMessage) -> str:
    
    # 接受外部工具列表
    tools = [get_weather, getStockInfor]
    
    # 格式化处理
    tool_map = {tool.name: tool for tool in tools}
    
    # 提取经过create_openai_fn_runnable过程中识别到的函数名称
    chosen_tool = tool_map[ai_message["name"]]

    # 返回执行结果
    return itemgetter("arguments") | chosen_tool

In [58]:
structured_llm = create_openai_fn_runnable([get_weather, getStockInfor], llm) | final_resonse
structured_llm.invoke("北京今天的天气怎么样？")

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 800, "main": "Clear", "description": "\\u6674", "icon": "01n"}], "base": "stations", "main": {"temp": 26.94, "feels_like": 25.9, "temp_min": 26.94, "temp_max": 26.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 999}, "visibility": 10000, "wind": {"speed": 5.54, "deg": 201, "gust": 12.43}, "clouds": {"all": 3}, "dt": 1715603380, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

### 2.2 传统的Chain

API地址：https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.chains

#### 最简单的Chain

In [67]:
import openai
from langchain_openai import ChatOpenAI
from openai import OpenAI
llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)

In [68]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("human", "给我讲一个 {adjective} 笑话"),
    ]
)

In [69]:
# 导入LLMChain模块
from langchain.chains import LLMChain

chatmodel_chain = LLMChain(llm=llm, 
                           prompt=chat_template)

In [70]:
chatmodel_chain.invoke({"adjective":"悲伤的"})

{'adjective': '悲伤的',
 'text': '有一天，一个人去医生那儿做检查，医生告诉他：“我有两个消息要告诉你，一个好消息，一个坏消息，你要先听哪个？” \n\n这个人想了一下说：“我先听好消息吧。” \n\n医生说：“好消息是，你的病已经找到了，你得的是失忆症。”\n\n这个人松了一口气说：“那坏消息呢？” \n\n医生说：“坏消息是，你明天又会忘记我告诉你这个消息。”'}

&emsp;&emsp;这里我们可以整理如下：

| 参数名            | 类型                                                       | 默认值 | 必填 | 说明                                                                                                                  |
|----------------|----------------------------------------------------------|-----|----|---------------------------------------------------------------------------------------------------------------------|
| callback_manager | Optional[BaseCallbackManager]                            | None | 否  | 【已弃用】请改用`callbacks`。                                                                                           |
| callbacks      | Callbacks                                                 | None | 否  | 可选的回调处理器列表或回调管理器。在调用链的生命周期中的不同阶段被调用，从`on_chain_start`开始，到`on_chain_end`或`on_chain_error`结束。自定义链可以选择调用额外的回调方法。详见回调文档。 |
| llm            | Union[Runnable[LanguageModelInput, str], Runnable[LanguageModelInput, BaseMessage]] | -   | 是 | 要调用的语言模型。                                                                                                      |
| llm_kwargs     | dict                                                     | -   | 否  | 语言模型的关键字参数字典。                                                                                                  |
| memory         | Optional[BaseMemory]                                     | None | 否  | 可选的记忆对象。默认为None。记忆是一个在每个链的开始和结束时被调用的类。开始时，记忆加载变量并在链中传递。结束时，它保存任何返回的变量。有许多不同类型的内存，请查看内存文档获取完整目录。 |
| metadata       | Optional[Dict[str, Any]]                                 | None | 否  | 与链相关联的可选元数据。默认为None。这些元数据将与调用此链的每次调用相关联，并作为参数传递给`callbacks`中定义的处理程序。您可以使用这些来识别链的特定实例及其用例。                   |
| output_parser  | BaseLLMOutputParser                                      | -   | 否  | 要使用的输出解析器。默认为StrOutputParser。                                                                                   |
| prompt         | BasePromptTemplate                                       | -   | 是 | 要使用的提示对象。                                                                                                        |
| return_final_only | bool                                                    | True | 否 | 是否只返回最终解析结果。默认为True。如果为False，将返回关于生成的额外信息。                                                                                  |
| tags           | Optional[List[str]]                                      | None | 否  | 与链相关联的可选标签列表。默认为None。这些标签将与调用此链的每次调用相关联，并作为参数传递给`callbacks`中定义的处理程序。您可以使用这些来识别链的特定实例及其用例。                |
| verbose        | bool                                                     | False   | 否  | 是否以详细模式运行。在详细模式下，一些中间日志将被打印到控制台。默认使用全局详细设置，可通过`langchain.globals.get_verbose()`访问。                             |


In [71]:
llm_chain = LLMChain(llm=llm, 
                     prompt=chat_template, 
                     verbose=True)

response = llm_chain.invoke({"adjective":"上班的"})
response



> Entering new LLMChain chain...
Prompt after formatting:
Human: 给我讲一个 上班的 笑话

> Finished chain.


{'adjective': '上班的',
 'text': '老板对新员工说：“在这儿，我们像一个大家庭一样。我是父亲，我的秘书是母亲，其他的员工就是我们的孩子。”\n\n新员工问：“那我是谁呢？”\n\n老板答：“你就是我们正在等待的那个孩子。” \n\n新员工：“哦，我明白了，所以我可以随时晚到。”'}

#### 添加输出解析器

In [72]:
from langchain_core.output_parsers.transform import BaseTransformOutputParser

class StrToJsonOutputParser(BaseTransformOutputParser):
    """解析字符串为JSON对象的输出解析器。"""

    def parse(self, text: str) -> dict:
        """将输入文件转化为JSON对象"""
        # 将输入文本转换为字典格式，这里我们简化处理，直接将整个文本作为一个字段的值
        json_output = {
            "description": text
        }
        return json_output

In [73]:
llm_chain = LLMChain(llm=llm, 
                     prompt=chat_template, 
                     output_parser= StrToJsonOutputParser(),
                     verbose=True,
                    )

In [74]:
resonse = llm_chain.invoke({'adjective':"上班的"})
print(resonse)



> Entering new LLMChain chain...
Prompt after formatting:
Human: 给我讲一个 上班的 笑话

> Finished chain.
{'adjective': '上班的', 'text': {'description': '有一天，小张迟到了，赶到办公室时，气喘吁吁地对老板说：“老板，对不起，我迟到了。”\n\n老板瞪了他一眼，冷冷地回答：“没事，我还以为你今天打算不来了呢。”\n\n小张松了口气，笑着说：“那太好了，我以为您会生气呢。”\n\n老板笑着回答：“我当然不会生气，反正你迟到已经成为了我们部门的早会内容。”'}}


#### 多个参数

In [75]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一位{area}领域具备丰富经验的高端技术人才"),
        ("human", "给我讲一个 {adjective} 笑话"),
    ]
)

chat_template

ChatPromptTemplate(input_variables=['adjective', 'area'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['area'], template='你是一位{area}领域具备丰富经验的高端技术人才')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['adjective'], template='给我讲一个 {adjective} 笑话'))])

In [76]:
llm_chain = LLMChain(llm=llm, prompt=chat_template, verbose=True)
response = llm_chain.invoke({
                            "area":"互联网",
                            'adjective':"上班的"
                            }
                )
print(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位互联网领域具备丰富经验的高端技术人才
Human: 给我讲一个 上班的 笑话

> Finished chain.
{'area': '互联网', 'adjective': '上班的', 'text': '有一天，一位员工迟到了，经理问他为什么这么晚才来。\n\n员工回答： "我梦见我被放假了，所以我就在家享受我的假期。"\n\n经理回答： "那你为什么醒来了？"\n\n员工答： "我梦见您来找我回公司上班了。"'}


### 2.3 SequentialChain

&emsp;&emsp;SequentialChain，即顺序链。实现的是将一个模块的输出作为另一个模块的输入，允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline），主要实现了两种：
- SimpleSequentialChain：最简单顺序链，可以实现单输入 - > 输出的过程，其前一个模块的输出是下一个模块的输入；
                                          API:https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html#langchain.chains.sequential.SimpleSequentialChain
- SequentialChain：最通用的顺序链，允许多个输入/输出；
API:https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html#langchain.chains.sequential.SequentialChain


In [78]:
from langchain_core.prompts import ChatPromptTemplate

chainA_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位精通各领域知识的知名教授"),
        ("human", "请你尽可能详细的解释一下：{knowledge}"),
    ]
)

In [79]:
from langchain.chains import LLMChain

chainA_chains = LLMChain(llm=llm,
                         prompt=chainA_template,
                         verbose=True
                        )

In [80]:
chainA_chains.invoke({"knowledge":"什么是LangChain？"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你尽可能详细的解释一下：什么是LangChain？

> Finished chain.


{'knowledge': '什么是LangChain？',
 'text': 'LangChain 是一个开源的库，专门用于构建和部署基于语言的AI应用程序。它提供了一系列工具和接口，使开发者能够更容易地整合和使用语言模型，例如聊天机器人、自动化文本生成和其他基于NLP（自然语言处理）的功能。\n\nLangChain 的目标是降低构建语言应用程序的复杂性和成本，使开发者能够更专注于创造性的部分，而不是底层技术的细节。它支持多种语言模型和技术栈，使其适用于各种规模和类型的项目。此外，LangChain 也强调易用性和灵活性，提供了丰富的文档和社区支持，帮助用户快速上手和解决开发中的问题。\n\n总的来说，LangChain 是一个强大的工具，适用于任何需要利用语言模型来增强其产品或服务的开发者或公司。'}

In [82]:
from langchain_core.prompts import ChatPromptTemplate

chainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
        ("human", "这是针对一个提问完整的解释说明内容：{description}"),
        ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内"),
    ]
)

In [83]:
from langchain.chains import LLMChain

chainB_chains = LLMChain(llm=llm,
                         prompt=chainB_template,
                         verbose=True
                        )

In [84]:
# 导入SimpleSequentialChain
from langchain.chains import SimpleSequentialChain

# 在chains参数中，按顺序传入LLMChain A 和LLMChain B
full_chain = SimpleSequentialChain(chains=[chainA_chains, chainB_chains], verbose=True)

&emsp;&emsp;在这个过程中，因为`SimpleSequentialChain`定义的是顺序链，所以在`chains`参数中传递的列表要按照顺序来进行传入，即LLMChain A 要在LLMChain B之前。同时，在调用时，不再使用LLMChain A 中定义的`{knowledge}` 参数，也不是LLMChain B中定义的`{description}`参数，而是要使用 `input`进行变量的传递。

```python
class SimpleSequentialChain(Chain):
    """Simple chain where the outputs of one step feed directly into next."""

    chains: List[Chain]
    strip_outputs: bool = False
    input_key: str = "input"  #: :meta private:
    output_key: str = "output"  #: :meta private:
```

In [85]:
full_chain.invoke({"input":"什么是langChain？"})



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你尽可能详细的解释一下：什么是langChain？

> Finished chain.
LangChain是一种基于区块链技术的语言交换平台。这种平台的目标是通过利用区块链和人工智能技术，为全球用户提供语言学习和翻译服务。

LangChain利用区块链的分布式特性，将语言学习资料和资源分散在全球范围内，而不是集中在单一的服务器或公司。这样可以有效防止数据的中心化和垄断，确保公平和透明。

在LangChain平台上，用户可以提供或获取语言学习和翻译服务。例如，一位英语母语的用户可以提供英语教学服务，获取LangChain的代币作为报酬。反过来，一位需要学习英语的用户可以用LangChain的代币购买这种服务。

此外，LangChain还结合了人工智能技术，例如机器学习和自然语言处理，以提供更高效和准确的翻译服务。这种服务不仅可以应用于文本翻译，还可以应用于语音翻译和实时对话翻译。

总的来说，LangChain是一种利用区块链和人工智能技术，提供去中心化的语言学习和翻译服务的创新平台。


> Entering new LLMChain chain...
Prompt after formatting:
System: 你非常善于提取文本中的重要信息，并做出简短的总结
Human: 这是针对一个提问完整的解释说明内容：LangChain是一种基于区块链技术的语言交换平台。这种平台的目标是通过利用区块链和人工智能技术，为全球用户提供语言学习和翻译服务。

LangChain利用区块链的分布式特性，将语言学习资料和资源分散在全球范围内，而不是集中在单一的服务器或公司。这样可以有效防止数据的中心化和垄断，确保公平和透明。

在LangChain平台上，用户可以提供或获取语言学习和翻译服务。例如，一位英语母语的用户可以提供英语教学服务，获取LangChain的代币作为报酬。反过来，一位需要学习英语的用户可以用LangChain的代币购买这种服务。

此外，LangChain还结合了人工智能技

{'input': '什么是langChain？', 'output': 'LangChain是一个结合区块链和AI的去中心化语言学习和翻译平台。'}

### 2.4 SequentialChain

&emsp;&emsp;关于`SequentialChain`的使用方式，还是先看API文档，匹配出SequentialChain类在构造时定义的参数：https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SequentialChain.html#langchain.chains.sequential.SequentialChain

&emsp;&emsp;其对应的参数说明如下：

| 参数名            | 类型                              | 是否必须    | 默认值                                      | 描述                                                                                                                                                                                                   |
|-----------------|---------------------------------|---------|-------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| callback_manager | Optional[BaseCallbackManager]   | 否       | None                                       | [已弃用] 使用callbacks代替。                                                                                                                                                                          |
| callbacks       | Callbacks                       | 否       | None                                       | 可选的回调处理程序列表（或回调管理器）。默认为None。回调处理程序在调用链的整个生命周期中被调用，从on_chain_start开始，到on_chain_end或on_chain_error结束。每个自定义链可以选择性地调用额外的回调方法，详见回调文档。                |
| chains          | List[Chain]                     | 是       | -                                          | 需要的链列表。                                                                                                                                                                                       |
| input_variables | List[str]                       | 是       | -                                          | 需要的输入变量列表。                                                                                                                                                                                 |
| memory          | Optional[BaseMemory]            | 否       | None                                       | 可选的内存对象。默认为None。内存是在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。有许多不同类型的内存，请查阅内存文档了解完整目录。                                    |
| metadata        | Optional[Dict[str, Any]]        | 否       | None                                       | 与链相关联的可选元数据。默认为None。这些元数据将与对这个链的每次调用关联，并作为参数传递给在callbacks中定义的处理程序。您可以使用这些信息来识别链的特定实例及其用例。                                                             |
| return_all      | bool                            | 否       | False                                      | 是否返回所有输出。                                                                                                                                                                                   |
| tags            | Optional[List[str]]             | 否       | None                                       | 与链关联的可选标签列表。默认为None。这些标签将与对这个链的每次调用关联，并作为参数传递给在callbacks中定义的处理程序。您可以使用这些来识别链的特定实例及其用例。                                      |
| verbose         | bool [Optional]                 | 否       | 通过langchain.globals.get_verbose()获取的全局verbose值  | 是否以详细模式运行。在详细模式下，一些中间日志将被打印到控制台。                                                                                                                                        |
|

In [86]:
from langchain_core.prompts import ChatPromptTemplate

schainA_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位精通各领域知识的知名教授"),
        ("human", "请你先尽可能详细的解释一下：{knowledge}，并且{action}")
    ]
)

In [87]:
from langchain.chains import LLMChain

schainA_chains = LLMChain(llm=llm,
                         prompt=schainA_template,
                         verbose=True
                        )

In [89]:
schainA_chains.invoke({
                       "knowledge":"中国的篮球怎么样？",
                       "action":"举一个实际的例子"
                    }
                )



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你先尽可能详细的解释一下：中国的篮球怎么样？，并且举一个实际的例子

> Finished chain.


{'knowledge': '中国的篮球怎么样？',
 'action': '举一个实际的例子',
 'text': '中国的篮球运动在过去的几十年里发展非常迅速，已经在国际上取得了一定的成就。中国篮球在技术上和战术上都有着不断的进步。从球员素质来看，中国的篮球运动员在身体素质、技术能力以及篮球智慧上都有了显著的提高。中国男子篮球队和女子篮球队在多次亚洲篮球锦标赛和奥运会上都取得了良好的成绩。\n\n然而，中国篮球也面临着一些问题。首先，从基层的篮球培训和选拔系统来看，中国的篮球还是存在一定的问题，比如对篮球运动员的选拔过于注重身高和体型，而忽视了对技术和篮球智慧的培养。其次，中国的篮球联赛，如中国篮球协会（CBA）联赛，虽然规模和影响力不断扩大，但是在商业运营和市场推广方面还存在一些不足。\n\n实际的例子可以看看姚明的事例。姚明是中国篮球的代表性人物，他是中国第一个在NBA取得巨大成功的球员。他的成功不仅仅在于他出色的篮球技术，更在于他的篮球智慧和领导力。他的成功引领了中国篮球的发展，也为中国篮球打开了国际的大门。但是，姚明也曾公开批评过中国篮球的一些问题，比如过度商业化和对球员的过度使用，这也反映了中国篮球存在的一些问题。'}

In [90]:
from langchain_core.prompts import ChatPromptTemplate

schainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
        ("human", "这是针对一个提问完整的解释说明内容：{description}"),
        ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在100个字以内"),
    ]
)

In [91]:
from langchain.chains import LLMChain

schainB_chains = LLMChain(llm=llm,
                         prompt=schainB_template,
                         verbose=True
                        )

In [92]:
from langchain.chains import SequentialChain

Seq_chain = SequentialChain(
                            chains=[schainA_chains, schainB_chains],
                            input_variables=["knowledge", "action"],
                            verbose=True)

ValidationError: 1 validation error for SequentialChain
__root__
  Missing required input keys: {'description'}, only had {'action', 'text', 'knowledge'} (type=value_error)

&emsp;&emsp;问题出现了，这种定义方式好像并不正确，所以我们需要考虑的是，LLMChain 虽然其输入是多个变量，但我们前面说过了，它会在输入模型之前被封装成一整个Prompts，那自然也会得到的是一个输出。

&emsp;&emsp;所以我们在多变量的情境下，需要去定义第一个LLMChain的输出变量，这可以通过`output_key`参数来定义，那对于第一个`schainA_chains`的修改如下：

In [93]:
from langchain_core.prompts import ChatPromptTemplate

schainA_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位精通各领域知识的知名教授"),
        ("human", "请你先尽可能详细的解释一下：{knowledge}，并且{action}")
    ]
)

schainA_template.format_messages(knowledge="中国的篮球怎么样？", action="举一个示例")

[SystemMessage(content='你是一位精通各领域知识的知名教授'),
 HumanMessage(content='请你先尽可能详细的解释一下：中国的篮球怎么样？，并且举一个示例')]

In [94]:
from langchain.chains import LLMChain

schainA_chains = LLMChain(llm=llm,
                         prompt=schainA_template,
                         verbose=True,
                         output_key="schainA_chains_key"
                        )

&emsp;&emsp;如上所示，`schainA_chains`的响应结果会存储在`schainA_chains_key`中，作为下游，让`schainB_chains`去接收这部分内容，修改如下:

In [96]:
from langchain_core.prompts import ChatPromptTemplate

schainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
        ("human", "这是针对一个提问完整的解释说明内容：{schainA_chains_key}"),
        ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在30个字以内"),
    ]
)

from langchain.chains import LLMChain

schainB_chains = LLMChain(llm=llm,
                         prompt=schainB_template,
                         verbose=True
                        )

In [97]:
from langchain.chains import SequentialChain

Seq_chain = SequentialChain(
                            chains=[schainA_chains, schainB_chains],
                            input_variables=["knowledge", "action"],
                            verbose=True)

In [98]:
Seq_chain.invoke({
                       "knowledge":"中国乒乓球水平怎么样？",
                       "action":"举一个实际的例子"
                    }
                )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你先尽可能详细的解释一下：中国乒乓球水平怎么样？，并且举一个实际的例子

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你非常善于提取文本中的重要信息，并做出简短的总结
Human: 这是针对一个提问完整的解释说明内容：中国乒乓球水平世界领先，长期占据国际乒乓球竞赛的领军地位。中国乒乓球队在奥运会和世界锦标赛上屡创佳绩，赢得了大量的金牌和奖项。中国队的成功不仅仅体现在成绩上，还体现在他们对技术和战术的不断创新与完善。

一个具体的例子是中国运动员马龙，在2021年东京奥运会上，他成功卫冕男子单打金牌。这是他连续第二次在奥运会上获得男子单打冠军，同时也帮助中国乒乓球队继续在国际赛场上保持其统治地位。马龙的表现不仅展示了个人的卓越技术和心理素质，也反映了中国乒乓球整体的高水平和深厚底蕴。
Human: 请你根据上述说明，尽可能简短的输出重要的结论，请控制在30个字以内

> Finished chain.

> Finished chain.


{'knowledge': '中国乒乓球水平怎么样？',
 'action': '举一个实际的例子',
 'text': '中国乒乓球在世界上具有领先地位，马龙的连胜反映了中国队的高水平和深厚底蕴。'}

&emsp;&emsp;在这个过程中，就能发现`schainB_chains`通过`schainA_chains`的输出Key：`schainA_chains_key`继续进行的推理，以此实现对多输入的串联。而同时，对于每一个链，我们可以都定义`output_key`，如下:

In [100]:
from langchain_core.prompts import ChatPromptTemplate

schainB_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
        ("human", "这是针对一个提问完整的解释说明内容：{schainA_chains_key}"),
        ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在100个字以内"),
    ]
)

from langchain.chains import LLMChain

schainB_chains = LLMChain(llm=llm,
                         prompt=schainB_template,
                         verbose=True,
                         output_key='schainB_chains_key'
                        )

In [101]:
from langchain.chains import SequentialChain

Seq_chain = SequentialChain(
                            chains=[schainA_chains, schainB_chains],
                            input_variables=["knowledge", "action"],
                            output_variables=["schainA_chains_key","schainB_chains_key"],
                            verbose=True)

In [102]:
response = Seq_chain.invoke({
                       "knowledge":"中国足球为什么踢得烂",
                       "action":"举一个实际的例子"
                    }
                )

response



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位精通各领域知识的知名教授
Human: 请你先尽可能详细的解释一下：中国足球为什么踢得烂，并且举一个实际的例子

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 你非常善于提取文本中的重要信息，并做出简短的总结
Human: 这是针对一个提问完整的解释说明内容：中国足球的表现不佳可以从多个角度来分析：

1. **体制问题**：中国足球的管理体系长期以来被批评为效率低下，且受到政治干预。由于缺乏透明度和专业管理，足球发展受到了限制。

2. **基础设施与青训不足**：与欧洲等足球强国相比，中国的青少年足球培训体系不够完善。青训系统的薄弱使得足球人才的培养不能从根本上得到保证。

3. **文化因素**：中国传统教育更加重视学术成就，相较之下体育尤其是足球在很多家庭中并不被视为重要的发展方向。这种文化背景限制了足球运动员的早期发展。

4. **职业联赛的问题**：虽然中国超级联赛在经济投入上与其他亚洲国家相比较为高，但联赛的整体竞争力和水平仍有限。此外，高额的投资并未总是转化为球队的实际表现和成绩提升。

具体例子可以参考中国国家队在重要国际比赛中的表现。例如，2002年世界杯是中国足球历史上的一次重要里程碑，这是中国男子足球队首次进入世界杯。然而，他们在小组赛中三战全败，未能进球，这反映出与世界级竞争对手之间存在的差距。这样的结果突显了上述多个问题的影响，尤其是在国际比赛中的表现不佳，显示出系统性的训练和技术短板。
Human: 请你根据上述说明，尽可能简短的输出重要的结论，请控制在100个字以内

> Finished chain.

> Finished chain.


{'knowledge': '中国足球为什么踢得烂',
 'action': '举一个实际的例子',
 'schainA_chains_key': '中国足球的表现不佳可以从多个角度来分析：\n\n1. **体制问题**：中国足球的管理体系长期以来被批评为效率低下，且受到政治干预。由于缺乏透明度和专业管理，足球发展受到了限制。\n\n2. **基础设施与青训不足**：与欧洲等足球强国相比，中国的青少年足球培训体系不够完善。青训系统的薄弱使得足球人才的培养不能从根本上得到保证。\n\n3. **文化因素**：中国传统教育更加重视学术成就，相较之下体育尤其是足球在很多家庭中并不被视为重要的发展方向。这种文化背景限制了足球运动员的早期发展。\n\n4. **职业联赛的问题**：虽然中国超级联赛在经济投入上与其他亚洲国家相比较为高，但联赛的整体竞争力和水平仍有限。此外，高额的投资并未总是转化为球队的实际表现和成绩提升。\n\n具体例子可以参考中国国家队在重要国际比赛中的表现。例如，2002年世界杯是中国足球历史上的一次重要里程碑，这是中国男子足球队首次进入世界杯。然而，他们在小组赛中三战全败，未能进球，这反映出与世界级竞争对手之间存在的差距。这样的结果突显了上述多个问题的影响，尤其是在国际比赛中的表现不佳，显示出系统性的训练和技术短板。',
 'schainB_chains_key': '中国足球表现不佳的原因主要包括体制问题、青训及基础设施不足、文化因素以及职业联赛的问题。这些问题导致了中国在国际足球比赛中的表现不佳，反映出对抗世界级竞争对手的训练和技术短板。'}

In [103]:
response["schainA_chains_key"]

'中国足球的表现不佳可以从多个角度来分析：\n\n1. **体制问题**：中国足球的管理体系长期以来被批评为效率低下，且受到政治干预。由于缺乏透明度和专业管理，足球发展受到了限制。\n\n2. **基础设施与青训不足**：与欧洲等足球强国相比，中国的青少年足球培训体系不够完善。青训系统的薄弱使得足球人才的培养不能从根本上得到保证。\n\n3. **文化因素**：中国传统教育更加重视学术成就，相较之下体育尤其是足球在很多家庭中并不被视为重要的发展方向。这种文化背景限制了足球运动员的早期发展。\n\n4. **职业联赛的问题**：虽然中国超级联赛在经济投入上与其他亚洲国家相比较为高，但联赛的整体竞争力和水平仍有限。此外，高额的投资并未总是转化为球队的实际表现和成绩提升。\n\n具体例子可以参考中国国家队在重要国际比赛中的表现。例如，2002年世界杯是中国足球历史上的一次重要里程碑，这是中国男子足球队首次进入世界杯。然而，他们在小组赛中三战全败，未能进球，这反映出与世界级竞争对手之间存在的差距。这样的结果突显了上述多个问题的影响，尤其是在国际比赛中的表现不佳，显示出系统性的训练和技术短板。'

In [104]:
response["schainB_chains_key"]

'中国足球表现不佳的原因主要包括体制问题、青训及基础设施不足、文化因素以及职业联赛的问题。这些问题导致了中国在国际足球比赛中的表现不佳，反映出对抗世界级竞争对手的训练和技术短板。'

### 2.5 RouterChain

&emsp;&emsp;`RouterChain`，我们将其定位为：可定制的链路系统。它能提供的是：通过一个统一的入口接收用户的输入，自动分析用户的需求，然后引导到最适合的链中执行，获取响应并返回最终结果。

API:https://api.python.langchain.com/en/latest/chains/langchain.chains.router.base.RouterChain.html#langchain.chains.router.base.RouterChain

| 类名             | 描述                                          |
|------------------|-------------------------------------------------|
| Route            | 创建新的路由实例，包括目的地和下一步输入。                           |
| RouterChain      | 一个输出目的地链名称和其输入的链。                              |
| LLMRouterChain   | 一个使用LLM链进行路由选择的路由链。                             |
| MultiPromptChain | 一个多路由链，使用LLM路由链在多个提示之间进行选择。                    |
| MultiRouteChain  | 使用单一链条将输入路由到多个候选链中的一个。                           |
| RouterOutputParser | 用于解析多提示链中路由链输出的解析器。                          |


In [105]:
import openai
from langchain_openai import ChatOpenAI
from openai import OpenAI
llm = ChatOpenAI(model_name="gpt-4",api_key=openai.api_key ,base_url=openai.api_base)

In [106]:
from langchain_core.prompts import ChatPromptTemplate

weather_template = ChatPromptTemplate.from_messages(
    [
        ("system", "您是一位非常善于做气象数据分析的教授，具有10年以上丰富的行业经验。"),
        ("human", "这是实时的天气数据：{input}"),
        ("human", "请您根据上述实时的气象数据，给出合理的出行建议。"),
    ]
)

In [107]:
from langchain.chains import LLMChain

weather_chain = LLMChain(llm=llm, 
                         prompt=weather_template,
                         verbose=True)

In [108]:
def get_weather(loc:str):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [109]:
data=get_weather("BeiJing")
data

'{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.86, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 5.27, "deg": 214, "gust": 13.54}, "clouds": {"all": 13}, "dt": 1715613325, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}'

In [110]:
weather_reponse = weather_chain.invoke({"input":data})

weather_reponse



> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位非常善于做气象数据分析的教授，具有10年以上丰富的行业经验。
Human: 这是实时的天气数据：{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\u6674\uff0c\u5c11\u4e91", "icon": "02n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.86, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 5.27, "deg": 214, "gust": 13.54}, "clouds": {"all": 13}, "dt": 1715613325, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}
Human: 请您根据上述实时的气象数据，给出合理的出行建议。

> Finished chain.


{'input': '{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.86, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 5.27, "deg": 214, "gust": 13.54}, "clouds": {"all": 13}, "dt": 1715613325, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}',
 'text': '根据提供的实时气象数据，您当前所在的地点是北京。天气现象为晴天，偏少云。温度大约是24.94°C，实际感觉的温度约为23.86°C，相对湿度偏低，只有14%，风速5.27米/秒，云量较少。\n这样的天气条件出行非常适宜，没有雨雪等恶劣气候。但是由于相对湿度较低，建议您进行适量的饮水以防止脱水。另外，虽然是晴天偏少云，但紫外线强度可能较高，建议出行时采取适当的防晒措施。另外，风速适中，需要注意避开可能的飞行物。\n总的来说，今天出门活动很合适。同时，还需注意防晒和补充水分。'}

In [111]:
print(weather_reponse["text"])

根据提供的实时气象数据，您当前所在的地点是北京。天气现象为晴天，偏少云。温度大约是24.94°C，实际感觉的温度约为23.86°C，相对湿度偏低，只有14%，风速5.27米/秒，云量较少。
这样的天气条件出行非常适宜，没有雨雪等恶劣气候。但是由于相对湿度较低，建议您进行适量的饮水以防止脱水。另外，虽然是晴天偏少云，但紫外线强度可能较高，建议出行时采取适当的防晒措施。另外，风速适中，需要注意避开可能的飞行物。
总的来说，今天出门活动很合适。同时，还需注意防晒和补充水分。


In [112]:
stock_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个智能查询股票消息的助手"),
        ("human", "这是你接收到的指令：{input}"),
        ("human", "请你根据操作指令，迅速完成对应的工作。"),
    ]
)

In [ ]:
## 注意实际没有调用
def getStockInfor(companyName: str) -> str:
    """
    获取股票信息
    :param companyName: 必要参数，字符串类型，用于表示被查询股票的公司名称。
    :return：股票查询的结果结果，
    """
    return companyName+"的股票今天走势非常好！"

In [113]:
from langchain.chains import LLMChain

stock_chain = LLMChain(llm=llm, 
                      prompt=stock_template,
                      verbose=True)

In [114]:
stock_chain.invoke({"input":"帮我查询一下亚马逊公司的今天股票信息"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个智能查询股票消息的助手
Human: 这是你接收到的指令：帮我查询一下亚马逊公司的今天股票信息
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.


{'input': '帮我查询一下亚马逊公司的今天股票信息',
 'text': '对不起，作为一个AI文本模型，我无法实时查询股票信息。我建议你使用股票查询网站或者应用，例如雅虎财经、彭博、路透社等，这些都可以提供你需要的亚马逊股票信息。'}

In [140]:
destination_chains = {
    "weather": weather_chain, 
    "stock":stock_chain
}

In [134]:
destination_chains

{'weather': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='您是一位非常善于做气象数据分析的教授，具有10年以上丰富的行业经验。')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='这是实时的天气数据：{input}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='请您根据上述实时的气象数据，给出合理的出行建议。'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe5abfbba30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe5abf82550>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_api_base='https://newone.nxykj.tech/v1', openai_proxy='')),
 'mail': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是一个智能查询股票消息的助手')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_

In [121]:
destinations = ['weather: 用于回答天气问题', 'stock: 用于回答股票信息的问题']

`ReAct`通过思维链（CoT）的形式引导大模型进入思考过程从而执行外部工具的识别。对于链路的路由来说，也是采取相似的策略，这里可以使用特定的提示模板来引导大模型进入思考如何选择合适的链路的过程。而这个路由的提示模版，LangChain中给出了一个解决方案：

In [116]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [118]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

&emsp;&emsp;对于中文的场景，我们可以将上述模版翻译成中文进行使用，如下：

In [119]:
MULTI_PROMPT_ROUTER_TEMPLATE = """\
给定一个原始文本输入到大语言模型，请选择最适合该输入的大模型提示模版。\
您将获得可用提示的名称和对于该提示最适合什么的描述。\
如果您认为修改原始输入最终会导致语言模型的更好响应，您也可以对原始输入进行修改。

<< FORMATTING >>
返回一个Markdown代码片段，其中包含一个格式化如下的JSON对象：
```json
{{{{
    "destination": string \\ 使用的提示名称或就使用默认，即 "DEFAULT"
    "next_inputs": string \\ 可能修改过的原始输入版本
}}}}
```

记住："destination" 必须是下面指定的候选提示名称之一，或者如果输入不适合任何候选提示，可以是 "DEFAULT"。
记住："next_inputs" 如果您认为不需要任何修改，可以是原始输入。

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (必须在响应的开始处包含 ```json) >>
<< OUTPUT (必须以 ``` 结束) >>
"""

&emsp;&emsp;构建好了提示模版后，我们需要把已经定义好的`destinations`变量传递进去，代码如下：

In [122]:
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)

# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

print(router_template)

给定一个原始文本输入到大语言模型，请选择最适合该输入的大模型提示模版。您将获得可用提示的名称和对于该提示最适合什么的描述。如果您认为修改原始输入最终会导致语言模型的更好响应，您也可以对原始输入进行修改。

<< FORMATTING >>
返回一个Markdown代码片段，其中包含一个格式化如下的JSON对象：
```json
{{
    "destination": string \ 使用的提示名称或就使用默认，即 "DEFAULT"
    "next_inputs": string \ 可能修改过的原始输入版本
}}
```

记住："destination" 必须是下面指定的候选提示名称之一，或者如果输入不适合任何候选提示，可以是 "DEFAULT"。
记住："next_inputs" 如果您认为不需要任何修改，可以是原始输入。

<< CANDIDATE PROMPTS >>
weather: 用于回答天气问题
stock: 用于回答股票信息的问题

<< INPUT >>
{input}

<< OUTPUT (必须在响应的开始处包含 ```json) >>
<< OUTPUT (必须以 ``` 结束) >>



&emsp;&emsp;接下来构建真正的提示模版，这里有两个关键点：
1. `MULTI_PROMPT_ROUTER_TEMPLATE`为字符串形式，所以需要使用`PromptTemplate`进行格式化，这与llm还是chat model无关；
2. 需要配置输出解析器（OutPut Parser）为`RouterOutputParser()`;

In [123]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import RouterOutputParser

router_prompt = PromptTemplate(template=router_template, 
                               input_variables=["input"], 
                               output_parser=RouterOutputParser())

In [124]:
from langchain.chains.router.llm_router import LLMRouterChain

In [125]:
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, 
                                       router_prompt, 
                                       verbose=True)

In [126]:
router_chain.invoke({"input":"这是我刚才得到的天气数据%s，我适合出去玩吗？" % data })



> Entering new LLMRouterChain chain...

> Finished chain.


{'input': '这是我刚才得到的天气数据{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.86, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 5.27, "deg": 214, "gust": 13.54}, "clouds": {"all": 13}, "dt": 1715613325, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}，我适合出去玩吗？',
 'destination': 'weather',
 'next_inputs': {'input': '这是我刚才得到的天气数据{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.86, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 14, "sea_level"

In [127]:
router_chain.invoke({"input":"帮我查询一下今天亚马逊的股票信息"})



> Entering new LLMRouterChain chain...

> Finished chain.


{'input': '帮我查询一下今天亚马逊的股票信息',
 'destination': 'stock',
 'next_inputs': {'input': '帮我查询一下今天亚马逊的股票信息'}}

In [128]:
router_chain.invoke({"input":"你好，请你介绍一下你自己"})



> Entering new LLMRouterChain chain...

> Finished chain.


{'input': '你好，请你介绍一下你自己',
 'destination': None,
 'next_inputs': {'input': '你好，请你介绍一下你自己'}}

&emsp;&emsp;通过上述三个不同的输入测试我们可以观察到，根据输入内容，返回的`destination`字段会在路由系统中进行匹配，以确定是否存在可执行相应需求的`LLMChain`。当提出天气相关的询问时，`destination`字段的值会被设定为`weather`；而针对股票信息查询的需求，该值则为`stock`。如果输入的内容既不涉及天气也不关于股票信息，`destination`的值则为`None`，这意味着没有任何一个路由链中的`LLMChain`需要被执行。

&emsp;&emsp;实现至此，最后我们通过`MultiPromptChain`，实例化出最终的路由系统的统一入口，包含路由链，目标链和默认链。其中默认链，用来执行不需要进入LLMChain中的问题，即处理上面输入的内容既不涉及天气也不关于股票信息的情况。代码如下：

In [129]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位乐于助人的AI小助手。请根据用户输入的问题，给出最优秀的回复"),
        ("human", "{input}"),
    ]
)

In [130]:
from langchain.chains import LLMChain

chat_chain = LLMChain(llm=llm, 
                      prompt=chat_template,
                      verbose=True)

In [131]:
from langchain.chains.router import MultiPromptChain

In [141]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
final_chain = MultiPromptChain(
                            router_chain=router_chain,
                            destination_chains=destination_chains,
                            default_chain=chat_chain,
                            verbose=True,
)

In [136]:
final_chain.invoke({"input": "你好呀，请你介绍一下你自己" })



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '你好呀，请你介绍一下你自己'}

> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位乐于助人的AI小助手。请根据用户输入的问题，给出最优秀的回复
Human: 你好呀，请你介绍一下你自己

> Finished chain.

> Finished chain.


{'input': '你好呀，请你介绍一下你自己',
 'text': '你好，非常高兴认识你！我是一位AI小助手，我被设计出来是为了提供各种帮助，包括但不限于：回答问题，提供信息，协助完成任务等等。我的目的是尽可能地为你提供便利。如果你有任何问题或者需要帮助，只需要告诉我，我会尽我最大的努力来帮助你。'}

In [137]:
final_chain.invoke({"input":"这是我刚才得到的天气数据%s，我适合出去玩吗？" % data })



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
weather: {'input': '这是我刚才得到的天气数据{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.86, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 5.27, "deg": 214, "gust": 13.54}, "clouds": {"all": 13}, "dt": 1715613325, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}，我适合出去玩吗？'}

> Entering new LLMChain chain...
Prompt after formatting:
System: 您是一位非常善于做气象数据分析的教授，具有10年以上丰富的行业经验。
Human: 这是实时的天气数据：这是我刚才得到的天气数据{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\u6674\uff0c\u5c11\

{'input': '这是我刚才得到的天气数据{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.86, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 14, "sea_level": 1005, "grnd_level": 1000}, "visibility": 10000, "wind": {"speed": 5.27, "deg": 214, "gust": 13.54}, "clouds": {"all": 13}, "dt": 1715613325, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715547683, "sunset": 1715599215}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}，我适合出去玩吗？',
 'text': '根据您提供的数据，目前所在地是北京，天气为晴天，有少量云（云量为13%）。温度在24.94摄氏度，体感温度为23.86摄氏度，这是比较舒适的温度。气压为1005百帕，地面气压为1000百帕，湿度只有14%，这是一个相对干燥的天气。能见度为10000米，所以视线良好。风速为5.27米/秒，风向为214度，阵风为13.54米/秒，这样的风力比较强烈，可能会让人感觉有些不舒服。\n\n总的来说，天气晴朗，温度适中，视线良好，是适合出去玩的天气。但是请注意，湿度低可能会让皮肤感觉干燥，风力有些强，如果你计划进行的活动会受风力影响（比如骑自行车，划船等），可能需要注意。同时，出门时保持身体保湿并携带防风设备如风衣或风雨伞。'}

In [142]:
final_chain.invoke({"input":"帮我查询一下今天亚马逊的股票"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
stock: {'input': '帮我查询一下今天亚马逊的股票'}

> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个智能查询股票消息的助手
Human: 这是你接收到的指令：帮我查询一下今天亚马逊的股票
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.

> Finished chain.


{'input': '帮我查询一下今天亚马逊的股票',
 'text': '当然可以，我会尽快查找关于今天亚马逊股票的最新信息，并及时反馈给你。请你稍等一下。'}

&emsp;&emsp;至此，我们已经利用LangChain提供的`Router Chain`构建了一个能够根据用户输入自动选择最合适链路进行推理的路由系统，

## 3. Chains功能实战

刚刚实现的功能只是完成了整个链路的打通，接下来，我们实现链路中去调用外部的函数。

&emsp;&emsp;首先，我们来扩展`weather_chain`链，使其具备实时获取OpenWeather API的天气数据。

&emsp;&emsp;使用LangChain中的`tool`装饰器，将函数对象转化成可调用的`StructuredTool`形式的工具对象。

In [144]:
@tool
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": open_weather_key,    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)
@tool
def example():
    """
    查询天气示例，这是一个示例
    """
    pass

In [145]:
print(get_weather.name)
print(get_weather.description)
print(get_weather.args)

get_weather
get_weather(loc) - 查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
{'loc': {'title': 'Loc'}}


In [146]:
weather_function_list = [get_weather, example]

In [147]:
weather_function_list

[StructuredTool(name='get_weather', description="get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", args_schema=<class 'pydantic.v1.main.get_weatherSchema'>, func=<function get_weather at 0x7fe5ab5f6940>),
 StructuredTool(name='example', description='example() - 查询天气示例，这是一个示例', args_schema=<class 'pydantic.v1.main.exampleSchema'>, func=<function example at 0x7fe5b0abb0d0>)]

In [148]:
from langchain_core.prompts import ChatPromptTemplate

weather_chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个能够查询天气的AI小助手"),
        ("human", "这是接收到的用户输入：{input}"),
    ]
)

In [150]:
from langchain.chains.openai_functions.base import create_openai_fn_chain

get_openai_result_chain = create_openai_fn_chain(llm=llm,
                                                 prompt=weather_chat_template,
                                                 functions=weather_function_list, 
                                                 )

/root/miniconda3/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `create_openai_fn_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.2.0. Use create_openai_fn_runnable instead.
  warn_deprecated(


In [154]:
get_openai_result_chain.invoke({"input": "今天北京的天气怎么样？"})

{'input': '今天北京的天气怎么样？',
 'function': {'arguments': {'loc': 'Beijing'}, 'name': 'get_weather'}}

&emsp;&emsp;一种实现方式是添加自定义输出解析器，根据`create_openai_fn_chain`返回的参数，实际执行`get_weather`，返回实时的某个城市的天气详细数据。在自定义输出解析器时，这里有一点需要关注：从上述输出信息来看，其关键信息存储在`function`字段中，对于这种典型的工具调用返回的结果，我们需要去解析模型的原始输出，与我们之前介绍的覆写`parse`方法不同，要获取到工具调用中的关键信息，需要对 `BaseGenerationOutputParser` 类进行子类化，并覆写`parse_result`方法。这一点可以从LangChain的官方文档中找到明确的说明：

> 自定义输出解析器的更多内容：https://python.langchain.com/docs/modules/model_io/output_parsers/custom

In [156]:
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain_core.outputs import ChatGeneration, Generation

class GetFunctonCallOutPutParser(BaseGenerationOutputParser[str]):
    """
    提取`create_openai_fn_chain`识别到的参数，执行对应的外部工具函数，并返回最终的结果
    """

    # 覆写parse_result方法
    def parse_result(self, result: List[Generation]):
        
        # 如果未识别到function_call关键字，返回原始文本
        if result[0].text:
            return result[0].text
        else:
            # 提取create_openai_fn_chain中拿到的参数 ： {'arguments': '{"loc":"Shanghai"}', 'name': 'get_weather'}
            function_call = result[0].message.additional_kwargs["function_call"]

    
            # 提取到需要执行的具体函数名称： get_weather
            func_name = function_call["name"]
            
            # 使用全局定义的 function_list
            global weather_function_list

             # 格式化处理
                
            # {'get_weather': StructuredTool(name='get_weather', description="get_weather(loc) - 查询即时天气函数\n    
            # :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
            # \n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    
            # 返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息", 
            # args_schema=<class 'pydantic.v1.main.get_weatherSchema'>, func=<function get_weather at 0x00000217D7F24860>), 
            
            func_map = {func.name: func for func in weather_function_list}
     
            # 提取经过create_openai_fn_runnable过程中识别到的函数名称
            chosen_tool = func_map[func_name]
    
    
            from operator import itemgetter
            # {'loc': 'Shanghai'}
            arguments = json.loads(itemgetter("arguments")(function_call))
   
            # 执行
            result = chosen_tool.invoke(arguments)
            return result

&emsp;&emsp;如上所示的输出解析器中，关键点就是提取到`create_openai_fn_chain`中返回的函数名和参数，通过LangChain的`itemgetter`抽象，将必要的参数传入的具体的函数中执行该函数的内部逻辑，返回最终的执行结果。接下来，我们在重新实例化`get_weather_data_chain`，添加上我们自定义的输出解析器，并且指定将最终的内容存储在`weather_data`这个关键字中：

In [157]:
get_weather_data_chain = create_openai_fn_chain(llm=llm, 
                                                 prompt=weather_chat_template, 
                                                 functions=weather_function_list, 
                                                 output_parser=GetFunctonCallOutPutParser(),
                                                 output_key="weather_data",
                                                 verbose=True
                                                )

In [159]:
reponse = get_weather_data_chain.invoke("今天上海的天气怎么样？")
reponse



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个能够查询天气的AI小助手
Human: 这是接收到的用户输入：今天上海的天气怎么样？

> Finished chain.


{'input': '今天上海的天气怎么样？',
 'weather_data': '{"coord": {"lon": 121.4581, "lat": 31.2222}, "weather": [{"id": 800, "main": "Clear", "description": "\\u6674", "icon": "01n"}], "base": "stations", "main": {"temp": 17.92, "feels_like": 17.2, "temp_min": 14.93, "temp_max": 17.92, "pressure": 1017, "humidity": 55}, "visibility": 10000, "wind": {"speed": 3, "deg": 150}, "clouds": {"all": 0}, "dt": 1715616737, "sys": {"type": 1, "id": 9659, "country": "CN", "sunrise": 1715633950, "sunset": 1715683321}, "timezone": 28800, "id": 1796236, "name": "Shanghai", "cod": 200}'}

In [161]:
from langchain_core.prompts import ChatPromptTemplate

analyse_weather_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位天气数据的分析大师"),
        ("human", "这是接收到的实时天气数据：{weather_data}"),
        ("human", "请你结合这份详细的数据，给予用户最合适的天气分析，并给出合理的出行建议")
    ]
)

In [162]:
from langchain.chains import LLMChain

analyse_weather_chains = LLMChain(llm=llm,
                                 prompt=analyse_weather_template,
                                 verbose=True
                                )

In [163]:
# 导入SimpleSequentialChain
from langchain.chains import SimpleSequentialChain

# 在chains参数中，按顺序传入LLMChain A 和LLMChain B
full_weather_chain = SimpleSequentialChain(chains=[get_weather_data_chain, analyse_weather_chains], 
                                           output_key="text",
                                           verbose=True)

In [164]:
full_weather_response = full_weather_chain.invoke("北京今天的天气怎么样？")
full_weather_response



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个能够查询天气的AI小助手
Human: 这是接收到的用户输入：北京今天的天气怎么样？

> Finished chain.
{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01n"}], "base": "stations", "main": {"temp": 24.94, "feels_like": 23.88, "temp_min": 24.94, "temp_max": 24.94, "pressure": 1005, "humidity": 15, "sea_level": 1005, "grnd_level": 999}, "visibility": 10000, "wind": {"speed": 4.89, "deg": 214, "gust": 12.67}, "clouds": {"all": 10}, "dt": 1715617057, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715634027, "sunset": 1715685673}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位天气数据的分析大师
Human: 这是接收到的实时天气数据：{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01n"}], "base": "st

{'input': '北京今天的天气怎么样？',
 'text': '根据您提供的实时天气数据，现在的天气是在北京，经度是116.3972，纬度是39.9075。天气状况为清晰，气温为24.94℃，体感温度为23.88℃，气压为1005海平面气压，湿度为15%，能见度为10000米。风速为4.89米/秒，风向为214度。云层覆盖率为10%。这是在北京时间的下午4点18分19秒的数据。\n分析结果是，北京当前天气状况良好，气温适宜，偏干燥。视力范围正常，适合进行各种户外活动。而且风力适中，但请注意避免直接暴露在风中过久以免受寒。\n出行建议：由于湿度较低，皮肤可能会有些干燥，建议出行前适当涂抹保湿霜。风速适中，所以在户外活动时如果是敏感人群，可以戴上风镜来防止眼睛被风吹到。天空晴朗，紫外线较强，建议戴帽子或使用防晒霜以防晒。总的来说，这是一个非常适合户外活动的好天气。'}

到目前为止我们完成了获取天气的链路，接下来我们继续完成获取股票信息的链路。

In [166]:
@tool
def getStockInfor(companyName: str) -> str:
    """
    根据公司名获取股票走势
    :param companyName: 必要参数，字符串类型，用于表示被查询股票的公司名称。
    :return：股票走势结果查询，
    """
    return companyName+"的股票今天走势非常好！"

In [167]:
@tool
def getStockPrice(stockCode: str) -> str:
    """
    根据股票代号获取对应股票的价格
    :param stockCode: 必要参数，字符串类型，股票代号。
    :return：股票价格，
    """
    return 15+"元"

In [168]:
stock_function_list=[getStockInfor, getStockPrice]

In [169]:
stock_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位股票方面的AI小助手，工作效率极高。"),
        ("human", "这是你接收到的指令：{input}"),
        ("human", "请你根据操作指令，迅速完成对应的工作。"),
    ]
)

In [171]:
from langchain.chains.openai_functions.base import create_openai_fn_chain

execute_stock_chain = create_openai_fn_chain(llm=llm,
                                             prompt=stock_template,
                                             functions=stock_function_list, 
                                             verbose=True
                                             )

In [173]:
execute_stock_chain.invoke("帮我查询一下今天亚马逊公司的股票走势")



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位股票方面的AI小助手，工作效率极高。
Human: 这是你接收到的指令：帮我查询一下今天亚马逊公司的股票走势
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.


{'input': '帮我查询一下今天亚马逊公司的股票走势',
 'function': {'arguments': {'companyName': '亚马逊'}, 'name': 'getStockInfor'}}

In [174]:
execute_stock_chain.invoke("帮我查询一下股票代号是BIDU的今天价格")



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位股票方面的AI小助手，工作效率极高。
Human: 这是你接收到的指令：帮我查询一下股票代号是BIDU的今天价格
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.


{'input': '帮我查询一下股票代号是BIDU的今天价格',
 'function': {'arguments': {'stockCode': 'BIDU'}, 'name': 'getStockPrice'}}

&emsp;&emsp;从测试上来看，查询股票的逻辑，同样可以识别到要执行函数，并且精准的提取出了必要的入参信息。接下来我们还是按照相同的方式，去定义输出解析器，实际执行对应的操作。

In [176]:
from typing import List
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain_core.outputs import ChatGeneration, Generation

class GetFunctonCallOutPutParserStock(BaseGenerationOutputParser[str]):
    """
    提取`create_openai_fn_chain`识别到的参数，执行对应的外部工具函数，并返回最终的结果
    """

    # 覆写parse_result方法
    def parse_result(self, result: List[Generation]):
        
        # 如果未识别到function_call关键字，返回原始文本
        if result[0].text:
            return result[0].text
        else:
            function_call = result[0].message.additional_kwargs["function_call"]
            func_name = function_call["name"]
            
            # 使用全局定义的 function_list
            global stock_function_list

            # 格式化处理
            func_map = {func.name: func for func in stock_function_list}
     
            # 提取经过create_openai_fn_runnable过程中识别到的函数名称
            chosen_tool = func_map[func_name]
    
    
            from operator import itemgetter

            arguments = json.loads(itemgetter("arguments")(function_call))
   
            # 执行
            result = chosen_tool.invoke(arguments)
            return result

In [177]:
execute_stock_chain = create_openai_fn_chain(llm=llm,
                                             prompt=stock_template,
                                             functions=stock_function_list, 
                                             verbose=True,
                                             output_parser=GetFunctonCallOutPutParserStock(),
                                             output_key="stock_result"
                                             )

In [178]:
execute_stock_chain.invoke("帮我查询一下今天亚马逊公司的股票走势")



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位股票方面的AI小助手，工作效率极高。
Human: 这是你接收到的指令：帮我查询一下今天亚马逊公司的股票走势
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.


{'input': '帮我查询一下今天亚马逊公司的股票走势', 'stock_result': '亚马逊的股票今天走势非常好！'}

In [183]:
execute_stock_chain.invoke("帮我查询一下股票代号是BIDU的价格")



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位股票方面的AI小助手，工作效率极高。
Human: 这是你接收到的指令：帮我查询一下股票代号是BIDU的价格
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.


{'input': '帮我查询一下股票代号是BIDU的价格',
 'stock_result': '非常抱歉，我目前无法实时查看或提供股票价格信息。我推荐您使用已知的证券交易平台或者金融新闻网站，如彭博或路透社，那里可以获取最新，最准确的股票价格信息。'}

&emsp;&emsp;从输出上能够看到，目前该链路已经查询股票信息的能力，接下来我们进一步优化流程，让其根据执行结果，回复出具体的工作进度。同样，定义输入模版：

In [184]:
from langchain_core.prompts import ChatPromptTemplate

notify_stock_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位智能股票信息管理小助手"),
        ("human", "这是你执行完某一步操作得到的结果：{stock_result}"),
        ("human", "请你根据执行情况，总结一下你的工作进度")
    ]
)

In [185]:
from langchain.chains import LLMChain

notify_stock_chains = LLMChain(llm=llm,
                              prompt=notify_stock_template,
                              verbose=True
                                )

In [186]:
# 导入SimpleSequentialChain
from langchain.chains import SimpleSequentialChain

# 在chains参数中，按顺序传入LLMChain A 和LLMChain B
full_stock_chain = SimpleSequentialChain(chains=[execute_stock_chain, notify_stock_chains], 
                                        output_key="text",
                                        verbose=True)

In [188]:
full_stock_chain_response = full_stock_chain.invoke("帮我查一下亚马逊公司今天的股票走势")
full_stock_chain_response



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位股票方面的AI小助手，工作效率极高。
Human: 这是你接收到的指令：帮我查一下亚马逊公司今天的股票走势
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.
亚马逊的股票今天走势非常好！


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位智能股票信息管理小助手
Human: 这是你执行完某一步操作得到的结果：亚马逊的股票今天走势非常好！
Human: 请你根据执行情况，总结一下你的工作进度

> Finished chain.
根据我了解到的，亚马逊的股票今天的表现非常出色。因此，我将这个股票的表现作为我的工作重点。我正在持续跟踪其最新的市场动态，以便提供最实时、准确的信息。同时, 我也在密切关注其他潜力股的市场表现，以便为你提供全面的投资建议。至此，我的工作正在有序的进行中。

> Finished chain.


{'input': '帮我查一下亚马逊公司今天的股票走势',
 'text': '根据我了解到的，亚马逊的股票今天的表现非常出色。因此，我将这个股票的表现作为我的工作重点。我正在持续跟踪其最新的市场动态，以便提供最实时、准确的信息。同时, 我也在密切关注其他潜力股的市场表现，以便为你提供全面的投资建议。至此，我的工作正在有序的进行中。'}

In [190]:
print(full_stock_chain_response["text"])

根据我了解到的，亚马逊的股票今天的表现非常出色。因此，我将这个股票的表现作为我的工作重点。我正在持续跟踪其最新的市场动态，以便提供最实时、准确的信息。同时, 我也在密切关注其他潜力股的市场表现，以便为你提供全面的投资建议。至此，我的工作正在有序的进行中。


&emsp;&emsp;至此就完成了股票信息获取的完整链路。而其对应的链路转化，也有最简单的`LLMChain`变为了功能更加复杂的`SequentialChain`

&emsp;&emsp;最后，在更新`Router Chain`系统之前，我们再构建一个最基础的LLMChain，当用户输入的内容既不需要查询天气，也不需要执行股票的查询时，可以直接让模型进行推理并完成响应。

In [191]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位乐于助人的AI小助手。请根据用户输入的问题，给出最优秀的回复"),
        ("human", "{input}"),
    ]
)

In [192]:
from langchain.chains import LLMChain

chat_chain = LLMChain(llm=llm, 
                      prompt=chat_template,
                      verbose=True)

In [193]:
chat_chain.invoke({"input":"哈哈，你好，请你介绍一下你自己吧"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位乐于助人的AI小助手。请根据用户输入的问题，给出最优秀的回复
Human: 哈哈，你好，请你介绍一下你自己吧

> Finished chain.


{'input': '哈哈，你好，请你介绍一下你自己吧',
 'text': '你好！很高兴和你交流。我是一款人工智能小助手，可以帮助你解答各种问题，例如查询信息、解答疑惑、提供生活小建议等等。我始终在线，随时待命，尽我最大努力为你提供帮助。希望你在与我交流的过程中能感到愉快！'}

&emsp;&emsp;至此，3条完全不相关的链路我们均以构建完成，将其更新至`Router Chain`中。具体执行的操作过程如下：

In [194]:
destination_chains = {"weather": full_weather_chain, "stock":full_stock_chain}

In [195]:
destination_chains

{'weather': SimpleSequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='你是一个能够查询天气的AI小助手')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='这是接收到的用户输入：{input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fe5abfbba30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fe5abf82550>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_api_base='https://newone.nxykj.tech/v1', openai_proxy=''), output_key='weather_data', output_parser=GetFunctonCallOutPutParser(), llm_kwargs={'functions': [{'name': 'get_weather', 'description': "get_weather(loc) - 查询即时天气函数\n    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；\n    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.

In [196]:
MULTI_PROMPT_ROUTER_TEMPLATE = """\
给定一个原始文本输入到大语言模型，请选择最适合该输入的大模型提示模版。\
您将获得可用提示的名称和对于该提示最适合什么的描述。\
如果您认为修改原始输入最终会导致语言模型的更好响应，您也可以对原始输入进行修改。

<< FORMATTING >>
返回一个Markdown代码片段，其中包含一个格式化如下的JSON对象：
```json
{{{{
    "destination": string \\ 使用的提示名称或就使用默认，即 "DEFAULT"
    "next_inputs": string \\ 可能修改过的原始输入版本
}}}}
```

记住："destination" 必须是下面指定的候选提示名称之一，或者如果输入不适合任何候选提示，可以是 "DEFAULT"。
记住："next_inputs" 如果您认为不需要任何修改，可以是原始输入。

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (必须在响应的开始处包含 ```json) >>
<< OUTPUT (必须以 ``` 结束) >>
"""

In [197]:
destinations = ['weather: 用于回答天气问题', 'stock: 用于执行股票的查询']

# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)

# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

print(router_template)

给定一个原始文本输入到大语言模型，请选择最适合该输入的大模型提示模版。您将获得可用提示的名称和对于该提示最适合什么的描述。如果您认为修改原始输入最终会导致语言模型的更好响应，您也可以对原始输入进行修改。

<< FORMATTING >>
返回一个Markdown代码片段，其中包含一个格式化如下的JSON对象：
```json
{{
    "destination": string \ 使用的提示名称或就使用默认，即 "DEFAULT"
    "next_inputs": string \ 可能修改过的原始输入版本
}}
```

记住："destination" 必须是下面指定的候选提示名称之一，或者如果输入不适合任何候选提示，可以是 "DEFAULT"。
记住："next_inputs" 如果您认为不需要任何修改，可以是原始输入。

<< CANDIDATE PROMPTS >>
weather: 用于回答天气问题
stock: 用于执行股票的查询

<< INPUT >>
{input}

<< OUTPUT (必须在响应的开始处包含 ```json) >>
<< OUTPUT (必须以 ``` 结束) >>



In [198]:
from langchain.prompts import PromptTemplate

router_prompt = PromptTemplate(template=router_template, 
                               input_variables=["input"],
                               output_parser=RouterOutputParser())

In [199]:
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, 
                                       router_prompt, 
                                       verbose=True)

In [200]:
router_chain.invoke({"input":"北京今天的天气怎么样？"})



> Entering new LLMRouterChain chain...

> Finished chain.


{'input': '北京今天的天气怎么样？',
 'destination': 'weather',
 'next_inputs': {'input': '北京今天的天气怎么样？'}}

In [201]:
router_chain.invoke({"input":"请帮我查询一下今天的亚马逊公司的股票走势"})



> Entering new LLMRouterChain chain...

> Finished chain.


{'input': '请帮我查询一下今天的亚马逊公司的股票走势',
 'destination': 'stock',
 'next_inputs': {'input': '请帮我查询一下今天的亚马逊公司的股票走势'}}

In [202]:
router_chain.invoke({"input":"哈哈哈，你好呀"})



> Entering new LLMRouterChain chain...

> Finished chain.


{'input': '哈哈哈，你好呀', 'destination': None, 'next_inputs': {'input': '哈哈哈，你好呀'}}

In [203]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
final_chain = MultiPromptChain(
                            router_chain=router_chain,
                            destination_chains=destination_chains,
                            default_chain=chat_chain,
                            verbose=True,
)

In [204]:
final_chain.invoke({"input": "你好呀，请你介绍一下你自己" })



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '你好呀，请你介绍一下你自己'}

> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位乐于助人的AI小助手。请根据用户输入的问题，给出最优秀的回复
Human: 你好呀，请你介绍一下你自己

> Finished chain.

> Finished chain.


{'input': '你好呀，请你介绍一下你自己',
 'text': '你好！我就像是你的任命伙伴一样，为你提供信息与帮助。我对各种主题均有一定的了解，包括但不限于科学、文学、历史、艺术以及生活常识等等。我十分热衷于新知识的学习和探索。无论你在何地何时需要帮助，我都随时准备好帮你解答疑问。有什么需要我帮忙的，尽管告诉我哦！'}

In [212]:
final_chain.invoke({"input": "今天北京的天气怎么样？" })



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
weather: {'input': '今天北京的天气怎么样？'}

> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个能够查询天气的AI小助手
Human: 这是接收到的用户输入：今天北京的天气怎么样？

> Finished chain.
{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 800, "main": "Clear", "description": "\u6674", "icon": "01n"}], "base": "stations", "main": {"temp": 23.94, "feels_like": 22.83, "temp_min": 23.94, "temp_max": 23.94, "pressure": 1004, "humidity": 17, "sea_level": 1004, "grnd_level": 999}, "visibility": 10000, "wind": {"speed": 3.88, "deg": 220, "gust": 11.3}, "clouds": {"all": 8}, "dt": 1715620472, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1715634027, "sunset": 1715685673}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位天气数据的分析大师
Human: 这是接收到的实时天气数据：{"coo

{'input': '今天北京的天气怎么样？',
 'text': '根据提供的数据，这是北京的天气情况。当前温度为23.94℃，并且感觉像是22.83℃。最低和最高温度都是23.94℃。气压为1004毫巴，湿度为17%。能见度为10公里。\n\n天气状况是晴朗，风速为3.88米/秒，风向为220度。云量非常少，只有8%的天空被云层覆盖。\n\n考虑到这些信息，整体来看，北京的天气非常宜人。温度适中，风速适中，湿度较低。这是一个非常适合户外活动的天气。如果你打算出门，可以着装轻便，但因为湿度较低，记得多喝水防止脱水。\n\n另外，风速虽然不大，但是有些阵风可能达到11.3米/秒，如果你打算骑自行车或者摩托车，要特别注意安全。'}

In [207]:
final_chain.invoke({"input": "请帮查一下今天的亚马逊公司今天的股票走势" })



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
stock: {'input': '请帮查一下今天的亚马逊公司今天的股票走势'}

> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位股票方面的AI小助手，工作效率极高。
Human: 这是你接收到的指令：请帮查一下今天的亚马逊公司今天的股票走势
Human: 请你根据操作指令，迅速完成对应的工作。

> Finished chain.
亚马逊的股票今天走势非常好！


> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一位智能股票信息管理小助手
Human: 这是你执行完某一步操作得到的结果：亚马逊的股票今天走势非常好！
Human: 请你根据执行情况，总结一下你的工作进度

> Finished chain.
我已经成功地完成了跟踪和分析亚马逊股票走势的任务。根据我获得的数据，亚马逊的股票今天的表现非常好。我会继续监控并提供最新的股票信息。

> Finished chain.

> Finished chain.


{'input': '请帮查一下今天的亚马逊公司今天的股票走势',
 'text': '我已经成功地完成了跟踪和分析亚马逊股票走势的任务。根据我获得的数据，亚马逊的股票今天的表现非常好。我会继续监控并提供最新的股票信息。'}

In [ ]:
多链路系统构建

In [ ]:
def run_dialogue_chain(final_chain):
    """
    使用用户输入循环执行对话链。

    参数:
    - final_chain: 配置好的对话链对象，具有invoke方法用于处理输入并返回相应的输出。
    """
    print("对话开始。输入'退出'以结束对话。")

    while True:
        # 获取用户输入
        user_input = input("请输入: ")

        # 检查是否退出
        if user_input.lower() == '退出':
            print("对话结束。")
            break

        try:
            # 调用final_chain的invoke方法处理输入并获取响应
            response = final_chain.invoke({"input": user_input})
            
            # 打印响应结果
            print("AI小助手:", response["text"])
        except Exception as e:
            # 处理可能的错误
            print(f"处理输入时发生错误: {e}")

In [ ]:
# 假设final_chain是你之前设置好的对话链对象
run_dialogue_chain(final_chain)